In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

# import display function to display dataframes
from IPython.display import display

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-14 00:37:05--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.009s  

2025-10-14 00:37:06 (8.15 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [3]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [5]:
# Data types
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
# Data preparation
# Check if the missing values are presented in the features.
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [7]:
# If there are missing values:
# For categorical features, replace them with 'NA' i.e., lead_source, industry, employment_status, location columns
# For numerical features, replace with with 0.0 i.e., annual_income columns
categorical_features_with_na = ['lead_source', 'industry', 'employment_status', 'location']

for column in categorical_features_with_na:
    df[column] = df[column].fillna('NA')

numerical_features_with_na = ['annual_income']
for column in numerical_features_with_na:
    df[column] = df[column].fillna(0.0)


In [22]:
# verify nulls
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [18]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score', ]
categorical = ['lead_source', 'industry', 'employment_status', 'location']

# Question 1

In [23]:
# What is the most frequent observation (mode) for the column industry?
df['industry'].mode()[0]

'retail'

# Question 2

In [24]:
# Create the correlation matrix for the numerical features of your dataset.
# In a correlation matrix, you compute the correlation coefficient between every pair of features.
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


# Split the data

In [25]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Make sure that the target value converted is not in your dataframe.
y_train = df_train['converted']
y_val = df_val['converted']
y_test = df_test['converted']

del df_train['converted']
del df_val['converted']
del df_test['converted']

print("df_train : ", len(df_train))
print("y_train : ", len(y_train))

print("df_val: ", len(df_val))
print("y_val: ", len(y_val))

print("df_test : ", len(df_test))
print("y_test :", len(y_test))


df_train :  876
y_train :  876
df_val:  293
y_val:  293
df_test :  293
y_test : 293


# Question 3

In [28]:
# Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

score = df_train[categorical].apply(mutual_info_converted_score)
score.sort_values(ascending=False)

round(score, 2)

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

# Question 4

In [33]:
# Now let's train a logistic regression.
# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
# Fit the model on the training dataset.
## To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

dv = DictVectorizer(sparse=False)
# create the training matrix
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

# create the model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

# Train the model
model.fit(X_train, y_train)

# create the validation matrix
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

# predict the validation set
y_pred = model.predict(X_val)

converted_decision = (y_pred >= 0.5)
accuracy = (converted_decision == y_val).mean()
print("Accuracy, rounded off : ", round(accuracy, 2))

Accuracy, rounded off :  0.7


# Question 5

In [49]:
# Let's find the least useful feature using the feature elimination technique.
# Train a model using the same features and parameters as in Q4 (without rounding).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which feature has the smallest difference ? Note: The difference doesn't have to be positive.
features = categorical + numerical
# original accuracy
original_accuracy = accuracy
feature_to_remove = None
min_difference = float('inf')
for feature in features:
    # create a list of features excluding the current feature
    features_subset = [f for f in features if f != feature]

    # create the training matrix without the current feature
    train_dict = df_train[features_subset].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    # create and train the model
    q5_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    q5_model.fit(X_train, y_train)

    # create the validation matrix without the current feature
    val_dict = df_val[features_subset].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # predict the validation set
    y_pred = q5_model.predict(X_val)

    converted_decision = (y_pred >= 0.5)
    accuracy = (converted_decision == y_val).mean()
    print(f"Accuracy without {feature} : ", accuracy)

    difference = np.abs(original_accuracy - accuracy)

    if difference < min_difference:
        min_difference = difference
        feature_to_remove = feature

print("----")
print("Feature with smallest difference: ", feature_to_remove)

Accuracy without lead_source :  0.7030716723549488
Accuracy without industry :  0.6996587030716723
Accuracy without employment_status :  0.6962457337883959
Accuracy without location :  0.7098976109215017
Accuracy without number_of_courses_viewed :  0.5563139931740614
Accuracy without annual_income :  0.8532423208191127
Accuracy without interaction_count :  0.5563139931740614
Accuracy without lead_score :  0.7064846416382252
----
Feature with smallest difference:  lead_score


# Question 6

In [31]:
# Now let's train a regularized logistic regression.
# Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
# Train models using all the features as in Q4.
# Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
# Which value of C gives the best accuracy on the validation dataset
results = []
dv = DictVectorizer(sparse=False)
for c in [0.01, 0.1, 1, 10, 100]:
    # create the training matrix
    train_dict = df_train[categorical + numerical].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    # create the model
    c_model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)

    # Train the model
    c_model.fit(X_train, y_train)

    # create the validation matrix
    val_dict = df_val[categorical + numerical].to_dict(orient='records')
    X_val = dv.transform(val_dict)

    # predict the validation set
    y_pred = c_model.predict(X_val)

    converted_decision = (y_pred >= 0.5)
    accuracy = (converted_decision == y_val).mean()
    results.append((c, round(accuracy, 3)))
    print(f"Accuracy with C={c} : ", round(accuracy, 3))

print("----")
print("'C' value goving best accuracy : ", max(results, key=lambda x: x[1])[0])

Accuracy with C=0.01 :  0.7
Accuracy with C=0.1 :  0.7
Accuracy with C=1 :  0.7
Accuracy with C=10 :  0.7
Accuracy with C=100 :  0.7
----
C value goving best accuracy :  0.01
